# 네이버 기사 크롤링
- https://news.naver.com/section/105
- thread 사용

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
article_list = []

def get_article(page):
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    
    driver.get("https://news.naver.com/section/10"+str(page))
    articles = driver.find_elements(By.CSS_SELECTOR, '#newsct li')
    
    for article in articles:
        try:
            tmp_elements = article.find_elements(By.CSS_SELECTOR, '.sa_text strong')
            if tmp_elements:
                title = tmp_elements[0].text
            else:
                tmp_elements2 = article.find_elements(By.CSS_SELECTOR, '.ss_text a')
                if tmp_elements2:
                    title = tmp_elements2[0].text
                else:
                    title = "해당 정보 없음"
                    
            article_list.append(title)
        except:
            print("에러 발생!")
    
    print("end :", page)
    driver.quit()

In [3]:
%%time
for page in range(0, 6):
    get_article(page)

end : 0
end : 1
end : 2
end : 3
end : 4
end : 5
CPU times: total: 1.31 s
Wall time: 1min 10s


In [4]:
len(article_list), article_list[:30]

(304,
 ['野, \'의원 아니라 요원\' 김용현 주장에 "\'바이든-날리면\'식 기만"',
  '軍 공지합동 실사격훈련…“압도적 힘으로 목표물 타격”',
  '돌아온 이진숙…"방문진 대신 재허가부터"',
  '北최고인민회의 "국방력 변화에 투자 집중"…김정은 불참',
  '김문수 46% vs 이재명 42%…與잠룡 중 김문수가 압도적 1위',
  '경찰청장 대행 "서부지법 사태, 폭동이라는 데 동의"',
  '혈세로 마라톤·볼링 지방의원들, 연수에 고가 등산복…25억 적발',
  '계엄 후, 與가 野에 오차범위 밖 앞선 건 처음… 수도권·충청 우위',
  '북한, 尹 구속 상세 보도…"초췌…망동 정당화 횡설수설"',
  '文 "헛된 망상에 헌정체제 뒤흔들고 국민 분열…정치 극단화"',
  "'이재명 대세론' 없는 설 명절, 왠지 불안불안한 민주당",
  '이재명 발목 잡는 키워드 넷, 선거법· 비호감· 카톡검열· 박스권',
  '잠룡 한동훈 복귀 2월 중순? 김문수·홍준표와 진검 승부',
  '이재명 발목잡는 또 다른 이재명… 호감·비호감 모두 1위',
  '與도 野도 ‘기승전 이재명’ [이런정치]',
  '여론조사 못 믿겠다? 결과에 대한 집착부터 버려야 한다',
  '“우파 된 결정적 이유가 뭐냐”…조영남 돌직구에 김흥국이 언급한 이 남자',
  '한동훈, 등판 고민?…김경율 "오늘 \'잘 지내죠\' 문자 왔다"',
  '폭동 후폭풍...극우유튜버·참가자 개싸움 "후원금 돌려달라"',
  '권성동, 이재명 `쥐`에 빗대 맹공격…"고양이 흉내내도 고양이 안된다"',
  '\'尹 체포 방해\' 경호차장 경찰 재출석…"기관단총 평시에도 배치"',
  '[속보]권성동 "이재명 \'흑묘백묘론\' 처음부터 끝까지 단 한 줄도 믿을 수 없어"',
  '권성동 "李, 정책 갈아엎어…스토킹 고백처럼 끔찍"',
  '권성동 "이재명, 쥐가 흉내낸다고 고양이 되나"',
  '한동훈 곧 재등판? 김경률 "오늘 연락받아...시기 고민하는 듯"',
  '[속보] 권성동, "이재

### pandas 사용

In [5]:
import threading
import pandas as pd

df = pd.DataFrame(columns=["category", "title"])

In [6]:
def get_article(page):
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    
    driver.get("https://news.naver.com/section/10"+str(page))
    category = driver.find_element(By.CSS_SELECTOR, '.ct_snb_h2_a').get_attribute("innerText")
    articles = driver.find_elements(By.CSS_SELECTOR, '#newsct li')
    
    for article in articles:
        try:
            tmp_elements = article.find_elements(By.CSS_SELECTOR, '.sa_text strong')
            if tmp_elements:
                title = tmp_elements[0].text
            else:
                tmp_elements2 = article.find_elements(By.CSS_SELECTOR, '.ss_text a')
                if tmp_elements2:
                    title = tmp_elements2[0].text
                else:
                    title = "해당 정보 없음"
                    
            article_list.append(title)
        except:
            print("에러 발생!")

        df.loc[len(df)] = {"category":category, "title":title,}
        
    print("end :", page+1)
    driver.quit()

In [7]:
%%time
for page in range(0, 6):
    get_article(page)

end : 1
end : 2
end : 3
end : 4
end : 5
end : 6
CPU times: total: 859 ms
Wall time: 52.4 s


In [8]:
df

,category,title
0,정치,"헌재 나온 尹, 출장 '머리손질' 받았다…법무부 ""국격 위해 허용"""
1,정치,"국방장관 대행 “적엔 두려움, 국민엔 신뢰”…혹한기 실사격 훈련 시찰"
2,정치,"김경수 “민주당, 여론 압도 못 해…한 사람 독주 허용 않아야”"
3,정치,"경찰청장 대행 ""서부지법 난입은 폭동…소요죄와 비슷"""
4,정치,"권영세, '부정선거론' 전한길 옹호…대변인 ""원칙적 말씀"""
...,...,...
299,IT/과학,"“갤럭시S25가 AI폰 대중화 이끌 것”…삼성의 엄청난 자신감, 시장 반응은"
300,IT/과학,이진숙 “2인체제 의결 문제없어”…지상파 재허가부터?
301,IT/과학,美빅테크 AI에 日손정의 자본 힘 합치는데…韓기업 돌파구 절실
302,IT/과학,[강민성의 헬스토리] 사라지지 않는 입냄새…제거하는 확실한 방법은?


In [9]:
df['category'].value_counts()

category
경제       52
IT/과학    52
생활/문화    51
정치       50
사회       50
세계       49
Name: count, dtype: int64